# PART 1 : Install Dependencies & Run Spark Session

In [1]:
# Install pyspark and import:
! pip install pyspark

import pyspark
from pyspark.ml.classification import LogisticRegression

from pyspark.sql.functions import when, col

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=b71f586876563d1b8a412b79e47c00b1b4dc8a2575e5a1db99c6d2107291a824
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Create the pyspark session:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark").getOrCreate()

# PART 2: Clone & Explore dataset

In [3]:
# Clone the diabetes dataset from the github repository; if you get error saying "Project-4 directory already exists, move to next cell":
! git clone https://github.com/elluis1001/Project-4/

Cloning into 'Project-4'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 112 (delta 51), reused 54 (delta 17), pack-reused 0
Receiving objects: 100% (112/112), 901.64 KiB | 5.67 MiB/s, done.
Resolving deltas: 100% (51/51), done.


In [4]:
# Check if the dataset exists:
! ls /content/Project-4/Luis/Dataset/

diabetes_prediction_dataset.csv  diabetes_test_dataset.csv


In [5]:
# Create spark dataframe:
df_diabetes_data = spark.read.csv("/content/Project-4/Luis/Dataset/diabetes_prediction_dataset.csv", header=True, inferSchema=True)

In [6]:
# Display the dataframe:
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|Female|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|Female|54.0|           0|            0|        No Info|27.32|        6.6|                 80|       0|
|  Male|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|Female|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|  Male|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|Female|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|Female|44.0|           0|            0|          never|19.31|  

In [7]:
# Show amount of rows:
df_diabetes_data.count()

100000

In [8]:
# Print the schema:
df_diabetes_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- diabetes: integer (nullable = true)



In [9]:
# Count the total no. of diabetic and non-diabetic class (values of 1 indicating the presence of diabetes and 0 indicating the absence of diabetes):
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('diabetes').count().show()

(100000, 9)
+--------+-----+
|diabetes|count|
+--------+-----+
|       1| 8500|
|       0|91500|
+--------+-----+



In [10]:
 # Count the total no. of gender types:
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(100000, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|58552|
| Other|   18|
|  Male|41430|
+------+-----+



In [11]:
# Get the summary statistics:
df_diabetes_data.describe().show()

+-------+------+-----------------+------------------+------------------+---------------+-----------------+------------------+-------------------+-------------------+
|summary|gender|              age|      hypertension|     heart_disease|smoking_history|              bmi|       HbA1c_level|blood_glucose_level|           diabetes|
+-------+------+-----------------+------------------+------------------+---------------+-----------------+------------------+-------------------+-------------------+
|  count|100000|           100000|            100000|            100000|         100000|           100000|            100000|             100000|             100000|
|   mean|  NULL|41.88585600000013|           0.07485|           0.03942|           NULL|27.32076709999422|5.5275069999983275|          138.05806|              0.085|
| stddev|  NULL|22.51683987161704|0.2631504702289171|0.1945930169980986|           NULL|6.636783416648357|1.0706720918835468|  40.70813604870383|0.27888308976661896|
|   

# PART 3: Data Cleaning & Preparation

In [12]:
# Check for null values:
for col in df_diabetes_data.columns:
  print(col + ":", df_diabetes_data[df_diabetes_data[col].isNull()].count())

gender: 0
age: 0
hypertension: 0
heart_disease: 0
smoking_history: 0
bmi: 0
HbA1c_level: 0
blood_glucose_level: 0
diabetes: 0


In [13]:
# Look for the unnecessary values present:
def count_zeros():
  columns_list = ["age", "bmi", "HbA1c_level", "blood_glucose_level"]
  for i in columns_list:
    print(i+":",df_diabetes_data[df_diabetes_data[i]==0].count())

In [14]:
count_zeros()

age: 0
bmi: 0
HbA1c_level: 0
blood_glucose_level: 0


In [15]:
# Drop the 'other' rows in the gender columns:
string_to_remove = "Other"
df_diabetes_data = df_diabetes_data[df_diabetes_data['Gender'] != string_to_remove]

In [16]:
# Count the total no. of gender types:
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(99982, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|58552|
|  Male|41430|
+------+-----+



In [17]:
# Count the total no. of smoker/non-smoker types:
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('smoking_history').count().show()

(99982, 9)
+---------------+-----+
|smoking_history|count|
+---------------+-----+
|    not current| 6439|
|         former| 9352|
|        No Info|35810|
|        current| 9286|
|          never|35092|
|           ever| 4003|
+---------------+-----+



In [18]:
# Drop the 'other' rows in the gender columns:
string_to_remove_1= "No Info"
df_diabetes_data = df_diabetes_data[df_diabetes_data['smoking_history'] != string_to_remove_1]

In [19]:
# Count the total no. of smoker/non-smoker types:
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('smoking_history').count().show()

(64172, 9)
+---------------+-----+
|smoking_history|count|
+---------------+-----+
|    not current| 6439|
|         former| 9352|
|        current| 9286|
|          never|35092|
|           ever| 4003|
+---------------+-----+



In [20]:
# Count the total no. of gender types:
print((df_diabetes_data.count(), len(df_diabetes_data.columns)))
df_diabetes_data.groupBy('gender').count().show()

(64172, 9)
+------+-----+
|gender|count|
+------+-----+
|Female|38852|
|  Male|25320|
+------+-----+



In [21]:
# Assign in the 'gender'column 'Female' = 0, and 'Male' = 1:
from pyspark.sql.functions import when, col
df_diabetes_data = df_diabetes_data.withColumn("gender",
    when(col("gender") == "Female", 0).
    when(col("gender") == "Male", 1).
    otherwise(col("gender"))
)
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|     0|80.0|           0|            1|          never|25.19|        6.6|                140|       0|
|     1|28.0|           0|            0|          never|27.32|        5.7|                158|       0|
|     0|36.0|           0|            0|        current|23.45|        5.0|                155|       0|
|     1|76.0|           1|            1|        current|20.14|        4.8|                155|       0|
|     0|20.0|           0|            0|          never|27.32|        6.6|                 85|       0|
|     0|44.0|           0|            0|          never|19.31|        6.5|                200|       1|
|     1|42.0|           0|            0|          never|33.64|  

In [22]:
# Assign in the 'smoking_history': "never" = 0, "ever" = 1, "not current" = 2, "current" = 3, "former" = 4:
df_diabetes_data = df_diabetes_data.withColumn("smoking_history",
    when(col("smoking_history") == "never", 0).
    when(col("smoking_history") == "ever", 1).
    when(col("smoking_history") == "not current", 2).
    when(col("smoking_history") == "current", 3).
    when(col("smoking_history") == "former", 4).
    otherwise(col("smoking_history"))
)
df_diabetes_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|     0|80.0|           0|            1|              0|25.19|        6.6|                140|       0|
|     1|28.0|           0|            0|              0|27.32|        5.7|                158|       0|
|     0|36.0|           0|            0|              3|23.45|        5.0|                155|       0|
|     1|76.0|           1|            1|              3|20.14|        4.8|                155|       0|
|     0|20.0|           0|            0|              0|27.32|        6.6|                 85|       0|
|     0|44.0|           0|            0|              0|19.31|        6.5|                200|       1|
|     1|42.0|           0|            0|              0|33.64|  

# PART 4: Correlation Analysis & Feature Selection

In [23]:
# Gender and smoking_history needs to be converted to float data type for model to work:
df_diabetes_data = df_diabetes_data.withColumn("gender", col("gender").cast('float'))
df_diabetes_data = df_diabetes_data.withColumn("smoking_history", col("smoking_history").cast('float'))
df_diabetes_data.show()


+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+
|   0.0|80.0|           0|            1|            0.0|25.19|        6.6|                140|       0|
|   1.0|28.0|           0|            0|            0.0|27.32|        5.7|                158|       0|
|   0.0|36.0|           0|            0|            3.0|23.45|        5.0|                155|       0|
|   1.0|76.0|           1|            1|            3.0|20.14|        4.8|                155|       0|
|   0.0|20.0|           0|            0|            0.0|27.32|        6.6|                 85|       0|
|   0.0|44.0|           0|            0|            0.0|19.31|        6.5|                200|       1|
|   1.0|42.0|           0|            0|            0.0|33.64|  

In [24]:
# Find the correlation among the set of input & output variables:
for i in df_diabetes_data.columns:
  print("Correlation to outcome for {} is {}".format(i, df_diabetes_data.stat.corr("diabetes",i)))

Correlation to outcome for gender is 0.05699689368565596
Correlation to outcome for age is 0.26084962459224337
Correlation to outcome for hypertension is 0.19222574901207254
Correlation to outcome for heart_disease is 0.16961397731730365
Correlation to outcome for smoking_history is 0.06472564826560573
Correlation to outcome for bmi is 0.20442115545137657
Correlation to outcome for HbA1c_level is 0.43889709468177335
Correlation to outcome for blood_glucose_level is 0.449697968864106
Correlation to outcome for diabetes is 1.0


In [25]:
# Feature selection:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['gender', 'age', 'hypertension', 'heart_disease',
                                         'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level'], outputCol='features')
output_data = assembler.transform(df_diabetes_data)

In [26]:
# Print the schema:
output_data.printSchema()

root
 |-- gender: float (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: float (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- diabetes: integer (nullable = true)
 |-- features: vector (nullable = true)



In [27]:
# Display dataframe:
output_data.show()

+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+--------------------+
|gender| age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|diabetes|            features|
+------+----+------------+-------------+---------------+-----+-----------+-------------------+--------+--------------------+
|   0.0|80.0|           0|            1|            0.0|25.19|        6.6|                140|       0|[0.0,80.0,0.0,1.0...|
|   1.0|28.0|           0|            0|            0.0|27.32|        5.7|                158|       0|[1.0,28.0,0.0,0.0...|
|   0.0|36.0|           0|            0|            3.0|23.45|        5.0|                155|       0|[0.0,36.0,0.0,0.0...|
|   1.0|76.0|           1|            1|            3.0|20.14|        4.8|                155|       0|[1.0,76.0,1.0,1.0...|
|   0.0|20.0|           0|            0|            0.0|27.32|        6.6|                 85|       0|(8,[1,5,6,7],[20....|


# PART 5: Split Dataset & Build the Model

In [28]:
# Create final data:
from pyspark.ml.classification import LogisticRegression

final_data = output_data.select('features','diabetes')

In [29]:
# Print schema of final data:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- diabetes: integer (nullable = true)



In [30]:
# Split the dataset ; build the model:
train, test = final_data.randomSplit([0.7, 0.3])
models = LogisticRegression(labelCol= 'diabetes')
model = models.fit(train)

In [31]:
# Summary of the model:
summary = model.summary
summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|           diabetes|         prediction|
+-------+-------------------+-------------------+
|  count|              45017|              45017|
|   mean|0.10946975587000467|  0.081302619010596|
| stddev| 0.3122311547686878|0.27330232782774494|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



# PART 6: Evaluate and Save the Model

In [32]:
# Import BinaryClassificationEvaluator:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

predictions = model.evaluate(test)

In [33]:
# Display predictions:
predictions.predictions.show(100)

+--------------------+--------+--------------------+--------------------+----------+
|            features|diabetes|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|(8,[1,5,6,7],[0.1...|       0|[6.62028879289234...|[0.99866872872989...|       0.0|
|(8,[1,5,6,7],[0.3...|       0|[7.04725213678943...|[0.99913096013092...|       0.0|
|(8,[1,5,6,7],[0.8...|       0|[5.22662849821061...|[0.99465709479345...|       0.0|
|(8,[1,5,6,7],[0.8...|       0|[5.81498817693441...|[0.99702635335772...|       0.0|
|(8,[1,5,6,7],[0.8...|       0|[7.22494740768205...|[0.99927233904071...|       0.0|
|(8,[1,5,6,7],[0.8...|       0|[14.4057851623874...|[0.99999944582522...|       0.0|
|(8,[1,5,6,7],[1.0...|       0|[6.29073809791215...|[0.99815003724764...|       0.0|
|(8,[1,5,6,7],[1.2...|       0|[12.7872754187789...|[0.99999720388111...|       0.0|
|(8,[1,5,6,7],[1.3...|       0|[8.38578945981901...|[0.9997719663

In [34]:
# How well the model works:
evaluator = BinaryClassificationEvaluator(rawPredictionCol= 'rawPrediction', labelCol='diabetes')
evaluator.evaluate(model.transform(test))

0.9558051123857909

In [35]:
# Save model:
model.save("model")

In [36]:
# Load saved model back to the environment:
from pyspark.ml.classification import LogisticRegressionModel

model = LogisticRegressionModel.load('model')

# PART 7: Prediction on New Data with the saved model

In [37]:
# Create a new spark dataframe:
test_df = spark.read.csv('/content/Project-4/Luis/Dataset/diabetes_test_dataset.csv', header=True, inferSchema=True)

In [38]:
# Assign in the 'gender'column 'Female' = 0, and 'Male' = 1:
from pyspark.sql.functions import when, col
test_df = test_df.withColumn("gender",
    when(col("gender") == "Female", 0).
    when(col("gender") == "Male", 1).
    otherwise(col("gender"))
)
test_df.show()

+------+---+------------+-------------+---------------+-----+-----------+-------------------+
|gender|age|hypertension|heart_disease|smoking_history|  bmi|HbA1c_level|blood_glucose_level|
+------+---+------------+-------------+---------------+-----+-----------+-------------------+
|     0| 80|           1|            1|              0|25.19|        6.6|                140|
|     1| 28|           0|            0|              0|27.32|        5.7|                158|
|     0| 36|           0|            0|              3|23.45|        5.0|                155|
|     1| 76|           1|            1|              3|20.14|        4.8|                155|
|     0| 20|           0|            0|              0|27.32|        6.6|                 85|
|     0| 44|           0|            0|              0|19.31|        6.5|                200|
|     1| 42|           0|            0|              0|33.64|        4.8|                145|
|     0| 32|           0|            0|              0|27.32

In [39]:
# Create an additional feature merged column:
test_data = assembler.transform(test_df)

In [40]:
# Print the schema:
test_data.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- features: vector (nullable = true)



In [41]:
# Use model to make predictions:
results = model.transform(test_data)
results.printSchema()

root
 |-- gender: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- smoking_history: integer (nullable = true)
 |-- bmi: double (nullable = true)
 |-- HbA1c_level: double (nullable = true)
 |-- blood_glucose_level: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [42]:
# Display the predictions:
results.select('features','prediction').show()

+--------------------+----------+
|            features|prediction|
+--------------------+----------+
|[0.0,80.0,1.0,1.0...|       1.0|
|[1.0,28.0,0.0,0.0...|       0.0|
|[0.0,36.0,0.0,0.0...|       0.0|
|[1.0,76.0,1.0,1.0...|       0.0|
|(8,[1,5,6,7],[20....|       0.0|
|(8,[1,5,6,7],[44....|       0.0|
|[1.0,42.0,0.0,0.0...|       0.0|
|(8,[1,5,6,7],[32....|       0.0|
+--------------------+----------+

